In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
with open('/Users/derrickwan/Desktop/newyork_data.json') as f:
  newyork_data = json.load(f)

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
neighborhoods_data[0]

{u'geometry': {u'coordinates': [-73.84720052054902, 40.89470517661],
  u'type': u'Point'},
 u'geometry_name': u'geom',
 u'id': u'nyu_2451_34572.1',
 u'properties': {u'annoangle': 0.0,
  u'annoline1': u'Wakefield',
  u'annoline2': None,
  u'annoline3': None,
  u'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  u'borough': u'Bronx',
  u'name': u'Wakefield',
  u'stacked': 1},
 u'type': u'Feature'}

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [11]:
#New York City coordinates
latitude = 40.730610
longitude =  -73.935242

In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [ ]:
# Use this again later
CLIENT_ID = 'BGDRMVIYQT05DSPBVHQBV0LAF54DOP3AAKSZAIIE1P21LDBQ' # your Foursquare ID
CLIENT_SECRET = 'SLNGBPQHQY4PNMJPTVVE5GFNCKNGAW50PA3Y5BNCQCNBMD5K' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [14]:
neighborhoods.loc[0, 'Neighborhood']

u'Wakefield'

In [15]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.8947051766, -73.8472005205.


In [23]:
# type your answer here
LIMIT = 100
radius = 750

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=O3MIXLO0F1UQFN3RFCXG2WOASJNB0WMBYA2UPOC0PZ20GA1F&client_secret=GLPHADBOLKCYLMKNC4NBDZM1FSR2UI0EO4CGT4GPX5SZYYGK&v=20180605&ll=40.8947051766,-73.8472005205&radius=750&limit=100'

In [24]:
results = requests.get(url).json()
results

{u'meta': {u'code': 200, u'requestId': u'5fde612e671d1b69219d8cdd'},
 u'response': {u'groups': [{u'items': [{u'reasons': {u'count': 0,
       u'items': [{u'reasonName': u'globalInteractionReason',
         u'summary': u'This spot is popular',
         u'type': u'general'}]},
      u'referralId': u'e-0-4c537892fd2ea593cb077a28-0',
      u'venue': {u'categories': [{u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/food/dessert_',
          u'suffix': u'.png'},
         u'id': u'4bf58dd8d48988d1d0941735',
         u'name': u'Dessert Shop',
         u'pluralName': u'Dessert Shops',
         u'primary': True,
         u'shortName': u'Desserts'}],
       u'id': u'4c537892fd2ea593cb077a28',
       u'location': {u'address': u'4120 Baychester Ave',
        u'cc': u'US',
        u'city': u'Bronx',
        u'country': u'United States',
        u'crossStreet': u'Edenwald & Bussing Ave',
        u'distance': 127,
        u'formattedAddress': [u'4120 Baychester Ave (Edenwald & Bussing Av

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lollipops Gelato,Dessert Shop,40.894123,-73.845892
1,Rite Aid,Pharmacy,40.896649,-73.844846
2,Jackie's West Indian Bakery,Caribbean Restaurant,40.889283,-73.843310
3,Carvel Ice Cream,Ice Cream Shop,40.890487,-73.848568
4,Walgreens,Pharmacy,40.896528,-73.844700


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [30]:
print(venues.shape)
venues.head()

(10150, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [31]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,30,30,30,30,30,30
Annadale,9,9,9,9,9,9
Arden Heights,4,4,4,4,4,4
Arlington,5,5,5,5,5,5
Arrochar,22,22,22,22,22,22
Arverne,20,20,20,20,20,20
Astoria,96,96,96,96,96,96
Astoria Heights,15,15,15,15,15,15
Auburndale,19,19,19,19,19,19


In [32]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 432 uniques categories.


In [33]:
# one hot encoding
ny_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Op

In [34]:
restaurants = venues[venues["Venue Category"].str.contains('Restaurant')]
restaurants.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
15,Co-op City,40.874294,-73.829939,Arby's,40.870411,-73.828606,Fast Food Restaurant
16,Co-op City,40.874294,-73.829939,Townhouse Restaurant,40.876086,-73.828868,Restaurant
20,Co-op City,40.874294,-73.829939,Kennedy's,40.876807,-73.829627,Fast Food Restaurant
26,Eastchester,40.887556,-73.827806,Fish & Ting,40.885656,-73.829197,Caribbean Restaurant
28,Eastchester,40.887556,-73.827806,Dyre Fish Market,40.889318,-73.831453,Seafood Restaurant


In [35]:
#this isn't used for the clustering, this is just to see the complete names of all the restaurants left 

rest_onehot = pd.get_dummies(restaurants[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
rest_onehot['Neighborhood'] = restaurants['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [rest_onehot.columns[-1]] + list(rest_onehot.columns[:-1])
rest_onehot = rest_onehot[fixed_columns]

rest_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Burmese Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Hunan Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Polish Restaurant,Puerto Rican Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Soba Restaurant,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
15,Co-op City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,Co-op City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20,Co-op City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26,Eastchester,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28,Eastchester,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
#this is to create an aggregate of the number of different cuisines each neighborhood has 

nyc_rest = restaurants.groupby('Neighborhood', as_index = False).count()
nyc_rest

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allerton,4,4,4,4,4,4
1,Annadale,1,1,1,1,1,1
2,Arrochar,5,5,5,5,5,5
3,Arverne,2,2,2,2,2,2
4,Astoria,36,36,36,36,36,36
5,Astoria Heights,3,3,3,3,3,3
6,Auburndale,5,5,5,5,5,5
7,Bath Beach,17,17,17,17,17,17
8,Battery Park City,6,6,6,6,6,6
9,Bay Ridge,25,25,25,25,25,25


In [37]:
nyc_rest = nyc_rest[['Neighborhood', 'Venue Category']]
nyc_rest.head()

,Neighborhood,Venue Category
0,Allerton,4
1,Annadale,1
2,Arrochar,5
3,Arverne,2
4,Astoria,36


In [38]:
# set number of clusters
kclusters = 4

nyc_cluster = nyc_rest['Venue Category']

nyc_cluster = nyc_cluster.to_frame()

nyc_cluster.head()

,Venue Category
0,4
1,1
2,5
3,2
4,36


In [39]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 3, 1, 1, 2, 1, 0], dtype=int32)

In [40]:
# create a new dataframe that includes the cluster
ny_merged = nyc_rest.copy()

# add clustering labels
ny_merged["Cluster Labels"] = kmeans.labels_

In [41]:
ny_merged.rename(columns={"Venue Category": "Number of Restaurants"}, inplace = True)

ny_merged.head()

,Neighborhood,Number of Restaurants,Cluster Labels
0,Allerton,4,1
1,Annadale,1,1
2,Arrochar,5,1
3,Arverne,2,1
4,Astoria,36,3


In [42]:
ny_merged = ny_merged.join(neighborhoods.set_index("Neighborhood"), on="Neighborhood")

print(ny_merged.shape)
ny_merged.head() # check the last columns!

(264, 6)


,Neighborhood,Number of Restaurants,Cluster Labels,Borough,Latitude,Longitude
0,Allerton,4,1,Bronx,40.865788,-73.859319
1,Annadale,1,1,Staten Island,40.538114,-74.178549
2,Arrochar,5,1,Staten Island,40.596313,-74.067124
3,Arverne,2,1,Queens,40.589144,-73.791992
4,Astoria,36,3,Queens,40.768509,-73.915654


In [43]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [44]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0]


,Neighborhood,Number of Restaurants,Cluster Labels,Borough,Latitude,Longitude
9,Bay Ridge,25,0,Brooklyn,40.625801,-74.030621
12,Bayside,26,0,Queens,40.766041,-73.774274
19,Belmont,30,0,Bronx,40.857277,-73.888452
28,Brooklyn Heights,23,0,Brooklyn,40.695864,-73.993782
34,Carnegie Hill,20,0,Manhattan,40.782683,-73.953256
35,Carroll Gardens,23,0,Brooklyn,40.680540,-73.994654
38,Chelsea,23,0,Manhattan,40.744035,-74.003116
38,Chelsea,23,0,Staten Island,40.594726,-74.189560
42,Civic Center,24,0,Manhattan,40.715229,-74.005415
46,Clinton,24,0,Manhattan,40.759101,-73.996119


In [45]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1]


,Neighborhood,Number of Restaurants,Cluster Labels,Borough,Latitude,Longitude
0,Allerton,4,1,Bronx,40.865788,-73.859319
1,Annadale,1,1,Staten Island,40.538114,-74.178549
2,Arrochar,5,1,Staten Island,40.596313,-74.067124
3,Arverne,2,1,Queens,40.589144,-73.791992
5,Astoria Heights,3,1,Queens,40.770317,-73.894680
6,Auburndale,5,1,Queens,40.761730,-73.791762
8,Battery Park City,6,1,Manhattan,40.711932,-74.016869
10,Bay Terrace,5,1,Queens,40.782843,-73.776802
10,Bay Terrace,5,1,Staten Island,40.553988,-74.139166
11,Baychester,3,1,Bronx,40.866858,-73.835798


In [46]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2]


,Neighborhood,Number of Restaurants,Cluster Labels,Borough,Latitude,Longitude
7,Bath Beach,17,2,Brooklyn,40.599519,-73.998752
20,Bensonhurst,12,2,Brooklyn,40.611009,-73.995180
22,Boerum Hill,16,2,Brooklyn,40.685683,-73.983748
25,Brighton Beach,13,2,Brooklyn,40.576825,-73.965094
30,Bulls Head,9,2,Staten Island,40.609592,-74.159409
31,Bushwick,18,2,Brooklyn,40.698116,-73.925258
37,Central Harlem,15,2,Manhattan,40.815976,-73.943211
50,College Point,12,2,Queens,40.784903,-73.843045
53,Concourse Village,10,2,Bronx,40.824780,-73.915847
56,Cypress Hills,11,2,Brooklyn,40.682391,-73.876616


In [47]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3]


,Neighborhood,Number of Restaurants,Cluster Labels,Borough,Latitude,Longitude
4,Astoria,36,3,Queens,40.768509,-73.915654
39,Chinatown,40,3,Manhattan,40.715618,-73.994279
47,Clinton Hill,36,3,Brooklyn,40.693229,-73.967843
67,East Village,36,3,Manhattan,40.727847,-73.982226
102,Greenwich Village,43,3,Manhattan,40.726933,-73.999914
117,Jackson Heights,36,3,Queens,40.751981,-73.882821
149,Midtown South,36,3,Manhattan,40.748510,-73.988713
158,Murray Hill,58,3,Manhattan,40.748303,-73.978332
158,Murray Hill,58,3,Queens,40.764126,-73.812763
235,Turtle Bay,35,3,Manhattan,40.752042,-73.967708
